# Import package


In [17]:
import requests
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from autograd import grad, jacobian
from scipy.optimize import line_search
import datetime
from typing import List

# Data import and processing


In [20]:
settledate = pd.to_datetime(datetime.date.today())
bonds = pd.DataFrame(
    json.loads(
        requests.get(
            "https://asx.api.markitdigital.com/asx-research/1.0/bonds/government/exchange-traded?height=179&width=690"
        ).content
    )["data"]["items"]
)
bonds["maturity"] = bonds["securityDescription"].str.extract(r"(\d{2}-\d{2}-\d{2})")[0]
bonds["maturity"] = pd.to_datetime(bonds["maturity"], format="%d-%m-%y")
bonds = bonds[["maturity", "couponPercent", "priceBid", "priceAsk"]]
bonds["couponPercent"] = bonds["couponPercent"].astype(float) / 100
bonds.dropna(inplace=True)
bonds.reset_index(drop=True, inplace=True)
bonds

,maturity,couponPercent,priceBid,priceAsk
0,2047-03-21,0.0300,75.500,82.00
1,2026-04-21,0.0425,100.780,101.00
2,2027-04-21,0.0475,102.820,103.17
3,2029-04-21,0.0325,97.000,99.82
4,2033-04-21,0.0450,103.120,104.00
5,2037-04-21,0.0375,94.100,97.30
6,2028-05-21,0.0225,97.730,98.00
7,2041-05-21,0.0275,79.110,84.19
8,2039-06-21,0.0325,86.500,93.00
9,2051-06-21,0.0175,53.125,54.80


# Support function


In [ ]:
def year_transform(date: pd.Timestamp) -> float:
    """Transform date to actual year"""
    if date == settledate:
        return 0
    years = date.year - settledate.year
    check_date = pd.Timestamp(
        year=date.year,
        month=settledate.month,
        day=settledate.day,
    )
    years += (date - check_date).days / abs(
        (
            check_date
            + pd.DateOffset(years=np.sign((date - check_date).days))
            - check_date
        ).days
    )
    return years


def coupon_date_generate(maturity: pd.Timestamp) -> np.ndarray:
    """Generate coupon dates"""
    coupon_dates = [maturity]
    while maturity - pd.DateOffset(months=6) > settledate:
        maturity -= pd.DateOffset(months=6)
        coupon_dates.append(maturity)
    return np.array(list(map(year_transform, coupon_dates[::-1])))

# Discount factor formula


In [ ]:
def discount_factor(params: np.ndarray, t: np.ndarray) -> np.ndarray:
    """Calculate discount factor"""
    f0, f1, f2, gamma = params
    return np.exp(
        -(
            f0 * t
            + f1 * (gamma - np.exp(-t / gamma) * gamma)
            + f2 * (gamma - np.exp(-t / gamma) * (t + gamma))
        )
    )

# Bond pricing formula

In [ ]:
def bond_valuation(params: np.ndarray, maturity: pd.Timestamp, coupon: float) -> float:
    """Calculate bond valuation"""
    t = coupon_date_generate(maturity)
    cf = np.ones_like(t) * coupon / 2
    cf[0] += 1
    return np.sum(cf * discount_factor(params, t))

# Loss function

In [ ]:
def loss_function(params: np.ndarray, maturity: pd.Timestamp, coupon: float, bid: float, ask:float) -> float:
    """Objective function for optimization"""
    bond_price = bond_valuation(params, maturity, coupon)
    return (max(0, bond_price - ask)/ask)**2 + (max(0, bid - bond_price)/bid)**2 